# CAP 6640 
# Script Assignment 1
# Jan 25, 2024

# Group 4
# Andres Graterol
#                   UCF ID: 4031393
# Zachary Lyons
#                   UCF ID: 4226832
# Christopher Hinkle
#                   UCF ID: 4038573
# Nicolas Leocadio
#                   UCF ID: 3791733

We start by reading our corpus data, which is a list of tech news articles found from kaggle

https://www.kaggle.com/datasets/jeet2016/us-financial-news-articles

There is a requirements.txt file with all the needed python modules, "pip install -r requirements.txt"

Old: Prior to running, download corpus from top right of link and then extract to a folder called "archive" in the same directory as this file
New: Extract archive.zip (its 1.5GB lol) to ./archive


------------- ./ScriptAssignment2.ipynb

------------- ./archive_usfinancial/*

In [1]:
import nltk
import re
nltk.download('punkt')  # one time execution if not cached yet
nltk.download('stopwords')  # one time execution if not cached yet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import os
import json
      
def read_data(archives_dir):
    corpus = []
    counter = 0
    # os.walk gets all the folders and paths in given directory
    for root, dirs, files in os.walk(archives_dir):
        for i, file in enumerate(files):
            # join path(root) and filename so that this can be computer-independent as long as names are the same
            # open in readonly mode, need encoding and ignore errors for weird characters in docs
            with open(os.path.join(root, file), 'r', encoding="utf8", errors='ignore') as f:
                # load json, can use strings for index from json tree
                # if data you want is inside of some structure, can use index like ['test1']['title']
                # https://devqa.io/python-parse-json/
                
                data = json.load(f)
                appendme = data['title'] + " " + data['published'] + " " + data['text'] # Create string with title, publish date, and document text
                
                if appendme: # Make sure not empty string
                    corpus.append((appendme))
            
            # This is here to only go through a few documents instead of the 2GB of documents
            # Leave in place to speed up testing/debug
            # Uncomment for real data/tests
            counter += 1
            if counter > 100: # Break because a lot of documents
                return corpus
            
# Extracted the articles into folder called "archive_usfinancial", change as needed
# read_data() will parse through all folders/files inside there

######################################################
########### WARNING: There are a lot of files in there and it took an hour and a half to extract on my desktop
######################################################
corpus = read_data(os.getcwd() + "\\archive_usfinancial")

In [3]:
def tokenize(corpus):
    regex = re.compile('[^a-zA-Z0-9]')  # regex to only match alphanumeric characters
    tokens = []
    
    for sent in nltk.sent_tokenize(corpus):
        for word in nltk.word_tokenize(sent):
            clean_word = regex.sub('', word)
            if clean_word:  # make sure we're not appending empty strings
                tokens.append(clean_word.lower())
    return tokens

tokens = tokenize(corpus[0])
tokens[50:60]

['the',
 'emerging',
 'market',
 'etf',
 'eem',
 'was',
 'a',
 'weak',
 'dollar',
 'and']

In [4]:
def remove_stopwords(tokens):
    stopwords = nltk.corpus.stopwords.words('english')
    return [word for word in tokens if word not in stopwords]

no_stopwords = remove_stopwords(tokens)
no_stopwords[50:60]

['charts',
 'dollartracking',
 'etf',
 'uup',
 'believes',
 'us',
 'currency',
 'continue',
 'help',
 'group']

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def stem_words(word):
    return stemmer.stem(word).strip()

stemmed = [stem_words(word) for word in no_stopwords]
stemmed[50:60]

['chart',
 'dollartrack',
 'etf',
 'uup',
 'believ',
 'us',
 'currenc',
 'continu',
 'help',
 'group']

In [6]:
# @TODO: Re-do the tfid vectorizer with the preprocessing as a single function
# @TODO: Maybe it's not needed though
# @TODO: same todo as below

# from nltk.stem.snowball import SnowballStemmer
# def preprocess_corpus(corpus):
#     # Tokenize
#     regex = re.compile('[^a-zA-Z0-9]')  # regex to only match alphanumeric characters
#     tokens = []
    
#     for sent in nltk.sent_tokenize(corpus):
#         for word in nltk.word_tokenize(sent):
#             clean_word = regex.sub('', word)
#             if clean_word:  # make sure we're not appending empty strings
#                 tokens.append(clean_word.lower())

#     print(tokens[50:60]) # Print for debug
    
#     # Remove stopwords
#     stopwords = nltk.corpus.stopwords.words('english')
#     no_stopwords = [word for word in tokens if word not in stopwords]

#     print(no_stopwords[50:60]) # Print for debug

#     # Stemming
#     stemmer = SnowballStemmer("english")

#     stemmed = [stemmer.stem(word).strip()for word in no_stopwords]
#     print(stemmed[50:60]) # Print for debug
        
#     return stemmed
     
# processed_data = preprocess_corpus(corpus[0])

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.95, max_features=50000,
                        min_df=0.0, stop_words='english',
                        use_idf=True, tokenizer=tokenize,
                        preprocessor=stem_words, lowercase=True)

tfidf_matrix = tfidf.fit_transform(corpus)

c:\Users\nick_\Documents\MSCpE\CAP6640 Natural Language Processing\venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\nick_\Documents\MSCpE\CAP6640 Natural Language Processing\venv\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', '

In [8]:
# @TODO: Re-do the tfid vectorizer with the preprocessing as a single function
# @TODO: Maybe it's not needed though

# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf2 = TfidfVectorizer(max_df=0.8, max_features=200000,
#                         min_df=0.05, stop_words='english',
#                         use_idf=True,
#                         preprocessor=preprocess_corpus, lowercase=True)
# tfidf_matrix2 = tfidf2.fit_transform(corpus)

In [9]:
# @TODO: Need to reformat and comment/write this better for readability and understanding
# @TODO: Also need to figure out what kind of results/metrics we can get out of this and report

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(tfidf_matrix)

for i,topic in enumerate(lda.components_):
    print(f'Topic #{i}:')
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

Topic #0:
['peso', '1', 'moneygram', 'company', 'stock', 'percent', 'said', 'exits', 'billion', 's']


Topic #1:
['futures', 'korea', 'macy', '2018', 'bitcoin', 'tax', 'said', 'year', 'percent', 's']


Topic #2:
['court', '2', 'iran', 'caterpillar', 'google', 'bitcoin', 'said', 'amazon', 'china', 's']


Topic #3:
['immigration', 'copper', 'ripple', 'commodity', 'said', 'gordon', '2018', '0054', 'prices', 's']


Topic #4:
['jones', 'cuban', 'tech', 'stocks', 'says', 'said', 'china', 'trump', 'cramer', 's']


